Code is from
https://github.com/warmspringwinds/tensorflow_notes/blob/master/tfrecords_guide.ipynb

In [1]:
import time

def time_usage(func):
    def wrapper(*args, **kwargs):
        beg_ts = time.time()
        retval = func(*args, **kwargs)
        end_ts = time.time() 
        print("elapsed time: %f" % (end_ts - beg_ts))
        return retval
    return wrapper

In [6]:
imgpath = "/home/walterms/project/walterms/mcmd/output/edgevar/imgs/"
# labels -- [iso, D, T, X, U, L]
# runs = {"edge15":}
runs = {"edge15.5": 1, "edge40": 0}
# runs = {"edge15": 1, "edge16": -1, "edge20": -1, 
#         "edge30": -1, "edge35": -1, "edge40": 0}
tfrecord_dir = "/home/walterms/project/walterms/mcmd/nn/cnn/tfrecords/"
tfrecord_list = [tfrecord_dir+run+".tfrecords" for run in runs]
print tfrecord_list

['/home/walterms/project/walterms/mcmd/nn/cnn/tfrecords/edge15.5.tfrecords', '/home/walterms/project/walterms/mcmd/nn/cnn/tfrecords/edge40.tfrecords']


In [5]:
import os
imgpath = "/home/walterms/project/walterms/mcmd/nn/cnn/data/benchmark/"
paths = [imgpath+x for x in os.listdir(imgpath)]
print fnames

['/home/walterms/project/walterms/mcmd/nn/cnn/data/benchmark/T_156.png', '/home/walterms/project/walterms/mcmd/nn/cnn/data/benchmark/edge10.png', '/home/walterms/project/walterms/mcmd/nn/cnn/data/benchmark/X_0.png', '/home/walterms/project/walterms/mcmd/nn/cnn/data/benchmark/X_1.png', '/home/walterms/project/walterms/mcmd/nn/cnn/data/benchmark/X_144.png', '/home/walterms/project/walterms/mcmd/nn/cnn/data/benchmark/T_1.png', '/home/walterms/project/walterms/mcmd/nn/cnn/data/benchmark/T_0.png', '/home/walterms/project/walterms/mcmd/nn/cnn/data/benchmark/edge15.png', '/home/walterms/project/walterms/mcmd/nn/cnn/data/benchmark/edge40.png']


In [7]:
writeRecords(imgpath,runs)

/home/walterms/project/walterms/mcmd/output/edgevar/imgs/edge15.5/500_999.tar.gz
/home/walterms/project/walterms/mcmd/output/edgevar/imgs/edge15.5/7000_7499.tar.gz
/home/walterms/project/walterms/mcmd/output/edgevar/imgs/edge15.5/5500_5999.tar.gz
/home/walterms/project/walterms/mcmd/output/edgevar/imgs/edge15.5/2500_2999.tar.gz
/home/walterms/project/walterms/mcmd/output/edgevar/imgs/edge15.5/7500_7999.tar.gz
/home/walterms/project/walterms/mcmd/output/edgevar/imgs/edge15.5/2000_2499.tar.gz
/home/walterms/project/walterms/mcmd/output/edgevar/imgs/edge15.5/5000_5499.tar.gz
/home/walterms/project/walterms/mcmd/output/edgevar/imgs/edge15.5/3000_3499.tar.gz
/home/walterms/project/walterms/mcmd/output/edgevar/imgs/edge15.5/8500_8999.tar.gz
/home/walterms/project/walterms/mcmd/output/edgevar/imgs/edge15.5/9500_9999.tar.gz
/home/walterms/project/walterms/mcmd/output/edgevar/imgs/edge15.5/8000_8499.tar.gz
/home/walterms/project/walterms/mcmd/output/edgevar/imgs/edge15.5/6000_6499.tar.gz
/home/

In [3]:
# Create TFRecord

# Important: We are using PIL to read .png files later.
# This was done on purpose to read indexed png files
# in a special way -- only indexes and not map the indexes
# to actual rgb values. This is specific to PASCAL VOC
# dataset data. If you don't want thit type of behaviour
# consider using skimage.io.imread()
from PIL import Image
import numpy as np
import tarfile
import os
import skimage.io as io
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

IMG_HEIGHT = 128
IMG_WIDTH = 128

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


# for img_path in filenames:
@time_usage
def writeRecords(imgpath, runs):
    for run in runs:
        tfrecord_filename = "tfrecords/"+run+".tfrecords"

        writer = tf.python_io.TFRecordWriter(tfrecord_filename)

        # Let's collect the real images to later on compare
        # to the reconstructed ones
        original_images = []

        zips = os.listdir(imgpath+run)
        for batch in zips:
            with tarfile.open(imgpath+run+"/"+batch,'r:gz') as tarf:
                cnt = 0
                print imgpath+run+"/"+batch
                for member in tarf:
                    cnt += 1
                    tar_img = tarf.extractfile(member)
                    img = np.array(Image.open(tar_img))
                    img_reduced = np.zeros(shape=(128,128,1), dtype=np.uint8)
                    for i in range(128):
                        for j in range(128):
                            img_reduced[i][j] = img[i][j][0]

                    img_raw = img_reduced.tostring()
    
                    if runs[run] == -1:
                        example = tf.train.Example(features=tf.train.Features(feature={
                            'height': _int64_feature(IMG_HEIGHT),
                            'width': _int64_feature(IMG_WIDTH),
                            'image_raw': _bytes_feature(img_raw)}))
                    else:
                        example = tf.train.Example(features=tf.train.Features(feature={
                            'height': _int64_feature(IMG_HEIGHT),
                            'width': _int64_feature(IMG_WIDTH),
                            'image_raw': _bytes_feature(img_raw),
                            'label': _int64_feature(runs[run])}))

                    writer.write(example.SerializeToString())

        writer.close()